## Q1

Q: Count all of the distinct ways to go up $n$ distinct steps one, two, or three steps at a time.

In [11]:
def stepUp(n):
    memo = [[], [[1]], [[1, 1], [2]], [[1, 1, 1], [1, 2], [2, 1], [3]]]
    i = 4
    while i <= n:
        memo.append([l + [1] for l in memo[i - 1]] + 
                    [l + [2] for l in memo[i - 2]] +
                    [l + [3] for l in memo[i - 3]])
        i += 1
    return memo[n]

In [13]:
import unittest

class TestStepUp(unittest.TestCase):
    # Base case tests.
    def testZero(self):
        self.assertEquals(stepUp(0), [])
    
    def testOne(self):
        self.assertEquals(stepUp(1), [[1]])        
    
    def testTwo(self):
        self.assertEquals(stepUp(2), [[1, 1], [2]])
    
    def testThree(self):
        self.assertEquals(stepUp(3), [[1, 1, 1], [1, 2], [2, 1], [3]])
        
    # Non base case test.
    def testFour(self):
        self.assertEquals(stepUp(4), [[1, 1, 1, 1], [1, 2, 1], [2, 1, 1], [3, 1], [1, 1, 2], [2, 2], [1, 3]])

if __name__ == '__main__':  unittest.main(argv=[''], exit=False)

/Users/alex/miniconda3/envs/data-structures-dev/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Please use assertEqual instead.
.....
----------------------------------------------------------------------
Ran 5 tests in 0.006s

OK


## Q2

Q: Find a path from the top left corner to the bottom right corner through a grid with optional obstacles throughout.

The solution is to memoize steps, and to attempt to always try to step in a defined order: (1) right (2) down, in some order of those actions being possible.

Note: I'm providing just a probably non-working implementation for this case, e.g. not testing this one.

In [ ]:
def findPath(arr):
    positions = [[0, 0]]
    dim_Y, dim_X = len(arr), len(arr[0])
    memo = [[None for _ in range(dim_X)] for _ in range(dim_Y)]
    while True:
        next_positions = []
        for pos in positions:
            if pos[0] + 1 < len(arr) and arr[pos[0]][pos[1]]:
                next_pos = [pos[0] + 1, pos[1]]
                if next_pos == [(len(arr) + 1)] * 2:
                    return memo[pos[0]][pos[1]] + next_pos
                else:
                    relevant_entry = memo[next_pos[0]][next_pos[1]]
                    new_route = memo[pos[0]][pos[1]] + [next_pos]
                    if relevant_entry:
                        relevant_entry.append(new_route)
                    else:
                        relevant_entry = [new_route]
            else:  # do the same for columns
                next_pos = [pos[0], pos[1] + 1]
                if next_pos == [(len(arr) + 1)] * 2:
                    return memo[pos[0]][pos[1]] + next_pos
                else:
                    relevant_entry = memo[next_pos[0]][next_pos[1]]
                    new_route = memo[pos[0]][pos[1]] + [next_pos]
                    if relevant_entry:
                        relevant_entry.append(new_route)
                    else:
                        relevant_entry = [new_route]                    